In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

# Data

In [ ]:
data_unfiltered = pd.read_excel("Your_Final_DataFrame")

data = data_unfiltered.dropna(subset=["Z1", "Z2", "label_grasp", "label_rotation"]).reset_index(drop=True)

Z1_values = pd.DataFrame(data["Z1"].apply(eval).tolist(), columns=[f"Z1_{i}" for i in range(12)])
Z2_values = pd.DataFrame(data["Z2"].apply(eval).tolist(), columns=[f"Z2_{i}" for i in range(12)])

X = pd.concat([Z1_values, Z2_values], axis=1).reset_index(drop=True)
y_grasp = data["label_grasp"].reset_index(drop=True)
y_rotation = data["label_rotation"].reset_index(drop=True)

valid_idx = X.notna().all(axis=1)
X = X[valid_idx].reset_index(drop=True)
y_grasp = y_grasp[valid_idx].reset_index(drop=True)
y_rotation = y_rotation[valid_idx].reset_index(drop=True)

assert len(X) == len(y_grasp) == len(
    y_rotation), f"Inconsistent lengths: X={len(X)}, y_grasp={len(y_grasp)}, y_rotation={len(y_rotation)}"

label_encoder_grasp = LabelEncoder()
label_encoder_rotation = LabelEncoder()

y_grasp_encoded = label_encoder_grasp.fit_transform(y_grasp)
y_rotation_encoded = label_encoder_rotation.fit_transform(y_rotation)

X_train, X_test, y_grasp_train, y_grasp_test, y_rotation_train, y_rotation_test = train_test_split(
    X, y_grasp_encoded, y_rotation_encoded, test_size=0.2, random_state=42, stratify=y_grasp_encoded
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

print("Распределение классов для захвата (label_grasp):")
print(y_grasp.value_counts().sort_index())

print("\nРаспределение классов для ротации (label_rotation):")
print(y_rotation.value_counts().sort_index())


# Random Forest

In [ ]:
model_rotation = RandomForestClassifier(n_estimators=100, random_state=42)
model_grasp = RandomForestClassifier(n_estimators=100, random_state=42)

model_rotation.fit(X_train, y_rotation_train)

model_grasp.fit(X_train, y_grasp_train)

y_grasp_pred = model_grasp.predict(X_test)
y_rotation_pred = model_rotation.predict(X_test)
y_grasp_pred_labels = label_encoder_grasp.inverse_transform(y_grasp_pred)
y_rotation_pred_labels = label_encoder_rotation.inverse_transform(y_rotation_pred)
target_names_grasp = ['Negative movement', 'No movement', 'Positive movement']
target_names_rotation = ['Negative movement', 'No movement', 'Positive movement']

accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")
print(classification_report(y_grasp_test, y_grasp_pred, target_names=target_names_grasp))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")
print(classification_report(y_rotation_test, y_rotation_pred, target_names=target_names_rotation))

# Weighted Random Forest

In [ ]:
model_rotation = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_grasp = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

model_rotation.fit(X_train, y_rotation_train)

model_grasp.fit(X_train, y_grasp_train)

y_grasp_pred = model_grasp.predict(X_test)
y_rotation_pred = model_rotation.predict(X_test)
y_grasp_pred_labels = label_encoder_grasp.inverse_transform(y_grasp_pred)
y_rotation_pred_labels = label_encoder_rotation.inverse_transform(y_rotation_pred)
target_names_grasp = ['Negative movement', 'No movement', 'Positive movement']
target_names_rotation = ['Negative movement', 'No movement', 'Positive movement']

accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")
print(classification_report(y_grasp_test, y_grasp_pred, target_names=target_names_grasp))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")
print(classification_report(y_rotation_test, y_rotation_pred, target_names=target_names_rotation))

# XGBoost

In [ ]:
model_rotation = XGBClassifier(n_estimators=100, random_state=42)
model_rotation.fit(X_train, y_rotation_train)

model_grasp = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model_grasp.fit(X_train, y_grasp_train)

y_grasp_pred = model_grasp.predict(X_test)
y_rotation_pred = model_rotation.predict(X_test)

unique_grasp_classes = np.unique(y_grasp_test)
unique_rotation_classes = np.unique(y_rotation_test)

target_names_grasp_filtered = label_encoder_grasp.inverse_transform(unique_grasp_classes)
target_names_rotation_filtered = label_encoder_rotation.inverse_transform(unique_rotation_classes)
 
accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")
print(classification_report(y_grasp_test, y_grasp_pred, labels=unique_grasp_classes, target_names=target_names_grasp_filtered))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")
print(classification_report(y_rotation_test, y_rotation_pred, labels=unique_rotation_classes, target_names=target_names_rotation_filtered))

# Weighted XGBoost

In [ ]:
unique_classes, class_counts = np.unique(y_rotation_train, return_counts=True)
total_samples = len(y_rotation_train)
class_weights = {cls: total_samples / count for cls, count in zip(unique_classes, class_counts)}
sample_weights_rotation = np.array([class_weights[label] for label in y_rotation_train])

model_rotation = XGBClassifier(n_estimators=100, random_state=42)
model_rotation.fit(X_train, y_rotation_train, sample_weight=sample_weights_rotation)

unique_classes, class_counts = np.unique(y_grasp_train, return_counts=True)
class_weights_grasp = {cls: total_samples / count for cls, count in zip(unique_classes, class_counts)}
sample_weights_grasp = np.array([class_weights_grasp[label] for label in y_grasp_train])

model_grasp = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model_grasp.fit(X_train, y_grasp_train, sample_weight=sample_weights_grasp)
y_grasp_pred = model_grasp.predict(X_test)
y_rotation_pred = model_rotation.predict(X_test)

unique_grasp_classes = np.unique(y_grasp_test)
unique_rotation_classes = np.unique(y_rotation_test)

target_names_grasp_filtered = label_encoder_grasp.inverse_transform(unique_grasp_classes)
target_names_rotation_filtered = label_encoder_rotation.inverse_transform(unique_rotation_classes)
 
accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")
print(classification_report(y_grasp_test, y_grasp_pred, labels=unique_grasp_classes, target_names=target_names_grasp_filtered))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")
print(classification_report(y_rotation_test, y_rotation_pred, labels=unique_rotation_classes, target_names=target_names_rotation_filtered))

# Logistic Regression and LightGBM

In [ ]:
model_logreg_grasp = LogisticRegression(max_iter=1000, random_state=42)
model_logreg_rotation = LogisticRegression(max_iter=1000, random_state=42)
model_logreg_grasp.fit(X_train, y_grasp_train)
model_logreg_rotation.fit(X_train, y_rotation_train)

model_lgbm_grasp = LGBMClassifier(n_estimators=100, random_state=42)
model_lgbm_rotation = LGBMClassifier(n_estimators=100, random_state=42)
model_lgbm_grasp.fit(X_train, y_grasp_train)
model_lgbm_rotation.fit(X_train, y_rotation_train)

y_grasp_pred_logreg = model_logreg_grasp.predict(X_test)
y_rotation_pred_logreg = model_logreg_rotation.predict(X_test)
y_grasp_pred_lgbm = model_lgbm_grasp.predict(X_test)
y_rotation_pred_lgbm = model_lgbm_rotation.predict(X_test)

for model_name, y_true, y_pred, label_encoder in [
    ('LogReg Grasp', y_grasp_test, y_grasp_pred_logreg, label_encoder_grasp),
    ('LogReg Rotation', y_rotation_test, y_rotation_pred_logreg, label_encoder_rotation),
    ('LGBM Grasp', y_grasp_test, y_grasp_pred_lgbm, label_encoder_grasp),
    ('LGBM Rotation', y_rotation_test, y_rotation_pred_lgbm, label_encoder_rotation)
]:
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\nТочность модели {model_name}: {accuracy:.2f}")
    print(classification_report(y_true, y_pred, target_names=['Negative movement', 'No movement', 'Positive movement']))


# Weighted Logistic Regression and LightGBM

In [ ]:
model_logreg_grasp = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model_logreg_rotation = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model_logreg_grasp.fit(X_train, y_grasp_train)
model_logreg_rotation.fit(X_train, y_rotation_train)

model_lgbm_grasp = LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_lgbm_rotation = LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_lgbm_grasp.fit(X_train, y_grasp_train)
model_lgbm_rotation.fit(X_train, y_rotation_train)

y_grasp_pred_logreg = model_logreg_grasp.predict(X_test)
y_rotation_pred_logreg = model_logreg_rotation.predict(X_test)
y_grasp_pred_lgbm = model_lgbm_grasp.predict(X_test)
y_rotation_pred_lgbm = model_lgbm_rotation.predict(X_test)

for model_name, y_true, y_pred, label_encoder in [
    ('LogReg Grasp', y_grasp_test, y_grasp_pred_logreg, label_encoder_grasp),
    ('LogReg Rotation', y_rotation_test, y_rotation_pred_logreg, label_encoder_rotation),
    ('LGBM Grasp', y_grasp_test, y_grasp_pred_lgbm, label_encoder_grasp),
    ('LGBM Rotation', y_rotation_test, y_rotation_pred_lgbm, label_encoder_rotation)
]:
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\nТочность {model_name}: {accuracy:.2f}")
    print(classification_report(y_true, y_pred, target_names=['Negative movement', 'No movement', 'Positive movement']))


# Weighted Models + FFT

### Random Forest

In [ ]:
X_train_fft = np.hstack([X_train.values, np.abs(np.fft.fft(X_train.values, axis=1))])
X_test_fft = np.hstack([X_test.values, np.abs(np.fft.fft(X_test.values, axis=1))])

In [ ]:
model_rotation = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_grasp = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

model_rotation.fit(X_train_fft, y_rotation_train)

model_grasp.fit(X_train_fft, y_grasp_train)

y_grasp_pred = model_grasp.predict(X_test_fft)
y_rotation_pred = model_rotation.predict(X_test_fft)
print("Уникальные классы для y_grasp:", np.unique(y_grasp))
print("Уникальные классы для y_rotation:", np.unique(y_rotation))
y_grasp_pred_labels = label_encoder_grasp.inverse_transform(y_grasp_pred)
y_rotation_pred_labels = label_encoder_rotation.inverse_transform(y_rotation_pred)
target_names_grasp = ['Negative movement', 'No movement', 'Positive movement']
target_names_rotation = ['Negative movement', 'No movement', 'Positive movement']

accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")
print(classification_report(y_grasp_test, y_grasp_pred, target_names=target_names_grasp))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")
print(classification_report(y_rotation_test, y_rotation_pred, target_names=target_names_rotation))


### XGBoost

In [ ]:
unique_classes, class_counts = np.unique(y_rotation_train, return_counts=True)
total_samples = len(y_rotation_train)

class_weights = {cls: total_samples / count for cls, count in zip(unique_classes, class_counts)}

sample_weights_rotation = np.array([class_weights[label] for label in y_rotation_train])

model_rotation = XGBClassifier(n_estimators=100, random_state=42)
model_rotation.fit(X_train_fft, y_rotation_train, sample_weight=sample_weights_rotation)

unique_classes, class_counts = np.unique(y_grasp_train, return_counts=True)
class_weights_grasp = {cls: total_samples / count for cls, count in zip(unique_classes, class_counts)}
sample_weights_grasp = np.array([class_weights_grasp[label] for label in y_grasp_train])

model_grasp = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model_grasp.fit(X_train_fft, y_grasp_train, sample_weight=sample_weights_grasp)
y_grasp_pred = model_grasp.predict(X_test_fft)
y_rotation_pred = model_rotation.predict(X_test_fft)

unique_grasp_classes = np.unique(y_grasp_test)
unique_rotation_classes = np.unique(y_rotation_test)

target_names_grasp_filtered = label_encoder_grasp.inverse_transform(unique_grasp_classes)
target_names_rotation_filtered = label_encoder_rotation.inverse_transform(unique_rotation_classes)

accuracy_grasp = accuracy_score(y_grasp_test, y_grasp_pred)
print(f"\nТочность модели для захвата: {accuracy_grasp:.2f}")

print(classification_report(y_grasp_test, y_grasp_pred, labels=unique_grasp_classes,
                            target_names=target_names_grasp_filtered))

accuracy_rotation = accuracy_score(y_rotation_test, y_rotation_pred)
print(f"\nТочность модели для ротации: {accuracy_rotation:.2f}")

print(classification_report(y_rotation_test, y_rotation_pred, labels=unique_rotation_classes,
                            target_names=target_names_rotation_filtered))

### Logistic Regression and Light GBM

In [ ]:
model_logreg_grasp = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model_logreg_rotation = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model_logreg_grasp.fit(X_train_fft, y_grasp_train)
model_logreg_rotation.fit(X_train_fft, y_rotation_train)

model_lgbm_grasp = LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_lgbm_rotation = LGBMClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_lgbm_grasp.fit(X_train_fft, y_grasp_train)
model_lgbm_rotation.fit(X_train_fft, y_rotation_train)

y_grasp_pred_logreg = model_logreg_grasp.predict(X_test_fft)
y_rotation_pred_logreg = model_logreg_rotation.predict(X_test_fft)
y_grasp_pred_lgbm = model_lgbm_grasp.predict(X_test_fft)
y_rotation_pred_lgbm = model_lgbm_rotation.predict(X_test_fft)

for model_name, y_true, y_pred, label_encoder in [
    ('LogReg Grasp', y_grasp_test, y_grasp_pred_logreg, label_encoder_grasp),
    ('LogReg Rotation', y_rotation_test, y_rotation_pred_logreg, label_encoder_rotation),
    ('LGBM Grasp', y_grasp_test, y_grasp_pred_lgbm, label_encoder_grasp),
    ('LGBM Rotation', y_rotation_test, y_rotation_pred_lgbm, label_encoder_rotation)
]:
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\nТочность модели {model_name}: {accuracy:.2f}")
    print(classification_report(y_true, y_pred, target_names=['Negative movement', 'No movement', 'Positive movement']))
